In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from datetime import datetime as dt
from mpl_toolkits.basemap import Basemap
%matplotlib inline

In [2]:
# load the google location history data
df_gps = pd.read_json('data/LocationHistory.json')
print 'There are {:,} rows'.format(len(df_gps))

There are 989,879 rows


In [3]:
# parse lat, lon, and timestamp from the dict inside the locations column
df_gps['lat'] = df_gps['locations'].map(lambda x: x['latitudeE7'])
df_gps['lon'] = df_gps['locations'].map(lambda x: x['longitudeE7'])
df_gps['timestamp_ms'] = df_gps['locations'].map(lambda x: x['timestampMs'])

# convert lat/lon to decimalized degrees and the timestamp to date-time
df_gps['lat'] = df_gps['lat'] / 10.**7
df_gps['lon'] = df_gps['lon'] / 10.**7
df_gps['timestamp_ms'] = df_gps['timestamp_ms'].astype(float) / 1000
df_gps['datetime'] = df_gps['timestamp_ms'].map(lambda x: dt.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [4]:
# drop columns we don't need, then show the dataframe
df_gps = df_gps.drop(labels=['locations', 'timestamp_ms'], axis=1, inplace=False)
df_gps.head()

,lat,lon,datetime
0,37.862431,-122.275535,2016-01-03 20:45:57
1,37.862542,-122.275483,2016-01-03 20:45:26
2,37.862431,-122.275535,2016-01-03 20:45:07
3,37.862431,-122.275535,2016-01-03 20:44:24
4,37.862431,-122.275535,2016-01-03 20:43:22


In [5]:
# define map colors
land_color = '#f5f5f3'
water_color = '#cdd2d4'
coastline_color = '#999999'
border_color = '#999999'
meridian_color = '#aaaaaa'
marker_fill_color = '#cc3300'
marker_edge_color = 'None'

In [6]:
# create the plot
fig = plt.figure(figsize=(20, 10))
fig.suptitle('GPS Location History', fontsize=30, y=0.95)
ax = fig.add_subplot(111, axisbg='#ffffff', frame_on=False)

# draw the basemap and its features
m = Basemap(projection='kav7', lon_0=0, resolution='l', area_thresh=10000)
m.drawmapboundary(color=border_color, fill_color=water_color)
m.drawcoastlines(color=coastline_color)
m.drawcountries(color=border_color)
m.fillcontinents(color=land_color, lake_color=water_color)
m.drawparallels(np.arange(-90., 120., 30.), color=meridian_color)
m.drawmeridians(np.arange(0., 420., 60.), color=meridian_color)

# project our points from each dataset then concatenate and scatter plot them
x, y = m(df_gps['lon'].values, df_gps['lat'].values)
m.scatter(x, y, s=20, color=marker_fill_color, edgecolor=marker_edge_color, alpha=0.9, zorder=3)

# show the map
plt.savefig('images/map-simple.png', dpi=300, bbox_inches='tight', pad_inches=0.2)
plt.show()

In [7]:
# first define a transverse mercator projection
map_width_m = 1000 * 1000
map_height_m = 1200 * 1000
target_crs = {'datum':'WGS84',
              'ellps':'WGS84',
              'proj':'tmerc',
              'lon_0':-119,
              'lat_0':37.5}

In [10]:
# plot the map
fig_width = 16
plt.figure(figsize=[fig_width, fig_width * map_height_m / float(map_width_m)])

m = Basemap(ellps=target_crs['ellps'],
            projection=target_crs['proj'],
            lon_0=target_crs['lon_0'], 
            lat_0=target_crs['lat_0'],
            width=map_width_m, 
            height=map_height_m,
            resolution='l',
            area_thresh=10000)

m.drawcoastlines(color=coastline_color)
m.drawcountries(color=border_color)
m.fillcontinents(color=land_color, lake_color=water_color)
m.drawstates(color=border_color)
m.drawmapboundary(fill_color=water_color)

x, y = m(df_gps['lon'].values, df_gps['lat'].values)
m.scatter(x, y, s=80, color='r', edgecolor='None', alpha=0.4, zorder=10)

plt.savefig('images/map-simple-ca.png', dpi=300, bbox_inches='tight', pad_inches=0.2)
plt.show()